<a href="https://colab.research.google.com/github/MihaelaCatan04/Sigmoid_Deep_Dive/blob/main/Lesson_2/Lesson_2_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Homework Nr. 2
#### Tasks:
1. Complete week 1 and 2 of second course from specialization;
2. Complete the practical Task
3. Watch the following videos, and try to integrate W&B in your code:
    https://www.youtube.com/watch?v=tHAFujRhZLA&t=207s
    
#### Practical Task:
Implement Neural Network using PyTorch to solve a task from selected Dataset.
Dataset can be one of the
1. https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset
2. https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data
3. Or other dataset that you can find interesting to solve with an NN

Steps:
1. Implement all needed libraries
2. Load and display few images from the dataset
3. Train-test split
4. Create DataLoaders for each set
5. Create Neural Network Class (multilayer)
6. Create Training and Testing loop
7. Train your model
8. Test your model
9. Output your results (accuracy and prediction examples)
10. IMPORTANT! Repeat this process with different: Activation functions, gradient descent techniques (SGD, Mini-batch GD), different hyperparameters (epocs, learning rate, ......)
11. Track your experiments, and show what was the combination that performed best, can you find the explanation for that?
11. Draw conclusions and see what are the differences

Good luck!

### Solution

Connect to google drive to save the dataset there.

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


Connect to Kaggle to download the datasete.

In [ ]:
!mkdir -p ~/.kaggle
!cp drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#download the dataset
!kaggle datasets download -d johnsmith88/heart-disease-dataset

Dataset URL: https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset
License(s): unknown
  0% 0.00/6.18k [00:00<?, ?B/s]
100% 6.18k/6.18k [00:00<00:00, 12.1MB/s]


In [ ]:
#unzip dataset
!unzip heart-disease-dataset.zip

Archive:  heart-disease-dataset.zip
  inflating: heart.csv               


Install wandb as part of the homework.

In [ ]:
!pip install wandb

Login to wandb.

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Import the necessary libraries.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import wandb

Load the dataset from kaggle and print its head using pandas.

In [ ]:
# Load dataset
data = pd.read_csv("heart.csv")

# Display data
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0


Split the dataset into train and test sets and normalize them because Neural Networks work only with noralized data.

In [ ]:
# Split dataset
X = data.drop("target", axis=1).values
y = data["target"].values

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Create a HeartDataset class that will transform dataframes in dataloaders.

In [ ]:
class HeartDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [ ]:
# DataLoaders
train_dataset = HeartDataset(X_train, y_train)
test_dataset = HeartDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Shape of dataloaders
for X, y in test_loader:
  print(f"Shape of X: {X.shape}")
  print(f"Shape of y: {y.shape}")
  break

Shape of X: torch.Size([32, 13])
Shape of y: torch.Size([32])


Set device to CUDA if available.

In [ ]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.mps.is_available()
    else "cpu"
)

print(f"Your laptop is using {device}")

Your laptop is using cpu


Create a Neural Network with:

*   Input layer: 13 neurons
*   1st layer: 128 neurons, activation function - tanh
*   2nd layer: 256 neurons, activation function - tanh
*   3rd layer: 512 neurons, activation function - tanh
*   Output layer: 1 neuron, activation function - Sigmoid



In [ ]:
class NeuralNetworkTanhSigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_tanh_sigmoid_stack = nn.Sequential(
            nn.Linear(13, 128),
            nn.Tanh(),
            nn.Linear(128, 256),
            nn.Tanh(),
            nn.Linear(256, 512),
            nn.Tanh(),
            nn.Linear(512, 1024),
            nn.Tanh(),
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        predictions = self.linear_tanh_sigmoid_stack(x)
        return predictions

model_tanh_sigmoid = NeuralNetworkTanhSigmoid().to(device)
print(model_tanh_sigmoid)

NeuralNetworkTanhSigmoid(
  (linear_tanh_sigmoid_stack): Sequential(
    (0): Linear(in_features=13, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): Tanh()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Tanh()
    (6): Linear(in_features=512, out_features=1024, bias=True)
    (7): Tanh()
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): Tanh()
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)


Create a Neural Network with:

*   Input layer: 13 neurons
*   1st layer: 128 neurons, activation function - Sigmoid
*   2nd layer: 256 neurons, activation function - Sigmoid
*   3rd layer: 512 neurons, activation function - Sigmoid
*   Output layer: 1 neuron, activation function - Sigmoid


In [ ]:
class NeuralNetworkSigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_sigmoid_stack = nn.Sequential(
            nn.Linear(13, 128),
            nn.Sigmoid(),
            nn.Linear(128, 256),
            nn.Sigmoid(),
            nn.Linear(256, 512),
            nn.Sigmoid(),
            nn.Linear(512, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 2048),
            nn.Sigmoid(),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        predictions = self.linear_sigmoid_stack(x)
        return predictions

model_sigmoid = NeuralNetworkSigmoid().to(device)
print(model_sigmoid)


NeuralNetworkSigmoid(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_sigmoid_stack): Sequential(
    (0): Linear(in_features=13, out_features=128, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=512, out_features=1024, bias=True)
    (7): Sigmoid()
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): Sigmoid()
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)


Create a Neural Network with:

*   Input layer: 13 neurons
*   1st layer: 128 neurons, activation function - ReLU
*   2nd layer: 256 neurons, activation function - ReLU
*   3rd layer: 512 neurons, activation function - ReLU
*   Output layer: 1 neuron, activation function - Sigmoid



In [ ]:
class NeuralNetworkReLUSigmoid(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_sigmoid_stack = nn.Sequential(
            nn.Linear(13, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 2048),
            nn.ReLU(),
            nn.Linear(2048, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.flatten(x)
        predictions = self.linear_relu_sigmoid_stack(x)
        return predictions

model_relu_sigmoid = NeuralNetworkReLUSigmoid().to(device)
print(model_relu_sigmoid)

NeuralNetworkReLUSigmoid(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_sigmoid_stack): Sequential(
    (0): Linear(in_features=13, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=1024, bias=True)
    (7): ReLU()
    (8): Linear(in_features=1024, out_features=2048, bias=True)
    (9): ReLU()
    (10): Linear(in_features=2048, out_features=1, bias=True)
    (11): Sigmoid()
  )
)


Define the train function.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    total_loss = 0

    for batch_no, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.unsqueeze(1).to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        if batch_no % 5 == 0 or batch_no == num_batches - 1:
            current = min((batch_no + 1) * len(X), size)
            print(f"Batch {batch_no + 1}/{num_batches} - Loss: {loss.item():.4f} [{current}/{size}]")

    avg_loss = total_loss / num_batches
    print(f"Epoch Training Loss: {avg_loss:.4f}")

Define the test function.

In [ ]:
def test(dataloader, model, loss_fn, epoch=None, log_to_wandb=False):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.unsqueeze(1).to(device)

            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += ((pred > 0.5) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    accuracy = correct / size

    if log_to_wandb and epoch is not None:
        wandb.log({
            "epoch": epoch + 1,
            "test_loss": test_loss,
            "test_accuracy": accuracy
        })

    print(f"Test Results:\n Accuracy: {(100 * accuracy):>0.1f}%, Avg Loss: {test_loss:.4f}\n")

Initialize a wandb run and save the configurations.

In [ ]:
wandb.init(
    project="neural-networks-homework",
    config={
        "dataset": "heart-disease-dataset",
        "epochs": 30,
        "batch_size": 32,
        "learning_rate": 0.001,
        "optimizer": "Adam"
    },
    settings=wandb.Settings(init_timeout=120)
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mihaelacatan (mihaela-catan). Use `wandb login --relogin` to force relogin


Run the 1st Neural Network:

In [ ]:
model = NeuralNetworkTanhSigmoid().to(device)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(model_tanh_sigmoid.parameters(), lr=wandb.config.learning_rate)

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch+1}\n------------------------------")
    train(train_loader, model, loss_func, optimizer)
    test(test_loader, model, loss_func)

torch.save(model.state_dict(), "NeuralNetworkTanhSigmoid.pth")
wandb.save("NeuralNetworkTanhSigmoid.pth")

Epoch 1
------------------------------
Batch 1/26 - Loss: 0.7004 [32/820]
Batch 6/26 - Loss: 0.6963 [192/820]
Batch 11/26 - Loss: 0.6993 [352/820]
Batch 16/26 - Loss: 0.6968 [512/820]
Batch 21/26 - Loss: 0.6978 [672/820]
Batch 26/26 - Loss: 0.7036 [520/820]
Epoch Training Loss: 0.6983
Test Results:
 Accuracy: 45.4%, Avg Loss: 0.6969

Epoch 2
------------------------------
Batch 1/26 - Loss: 0.6926 [32/820]
Batch 6/26 - Loss: 0.7008 [192/820]
Batch 11/26 - Loss: 0.6977 [352/820]
Batch 16/26 - Loss: 0.7008 [512/820]
Batch 21/26 - Loss: 0.7037 [672/820]
Batch 26/26 - Loss: 0.6979 [520/820]
Epoch Training Loss: 0.6982
Test Results:
 Accuracy: 45.4%, Avg Loss: 0.6969

Epoch 3
------------------------------
Batch 1/26 - Loss: 0.7028 [32/820]
Batch 6/26 - Loss: 0.6945 [192/820]
Batch 11/26 - Loss: 0.6932 [352/820]
Batch 16/26 - Loss: 0.7017 [512/820]
Batch 21/26 - Loss: 0.6987 [672/820]
Batch 26/26 - Loss: 0.6945 [520/820]
Epoch Training Loss: 0.6981
Test Results:
 Accuracy: 45.4%, Avg Loss: 

['/content/wandb/run-20241122_044045-u8jaqks4/files/NeuralNetworkTanhSigmoid.pth']

Run the 2nd Neural Network:

In [ ]:
model = NeuralNetworkSigmoid().to(device)
optimizer = torch.optim.Adam(model_sigmoid.parameters(), lr=wandb.config.learning_rate)

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch+1}\n------------------------------")
    train(train_loader, model, loss_func, optimizer)
    test(test_loader, model, loss_func)

torch.save(model.state_dict(), "NeuralNetworkSigmoid.pth")
wandb.save("NeuralNetworkSigmoid.pth")

Epoch 1
------------------------------
Batch 1/26 - Loss: 0.7138 [32/820]
Batch 6/26 - Loss: 0.6915 [192/820]
Batch 11/26 - Loss: 0.6915 [352/820]
Batch 16/26 - Loss: 0.6915 [512/820]
Batch 21/26 - Loss: 0.6915 [672/820]
Batch 26/26 - Loss: 0.6938 [520/820]
Epoch Training Loss: 0.6949
Test Results:
 Accuracy: 49.8%, Avg Loss: 0.6937

Epoch 2
------------------------------
Batch 1/26 - Loss: 0.6938 [32/820]
Batch 6/26 - Loss: 0.6960 [192/820]
Batch 11/26 - Loss: 0.6982 [352/820]
Batch 16/26 - Loss: 0.6982 [512/820]
Batch 21/26 - Loss: 0.6938 [672/820]
Batch 26/26 - Loss: 0.6867 [520/820]
Epoch Training Loss: 0.6948
Test Results:
 Accuracy: 49.8%, Avg Loss: 0.6937

Epoch 3
------------------------------
Batch 1/26 - Loss: 0.6916 [32/820]
Batch 6/26 - Loss: 0.7049 [192/820]
Batch 11/26 - Loss: 0.7071 [352/820]
Batch 16/26 - Loss: 0.6916 [512/820]
Batch 21/26 - Loss: 0.6982 [672/820]
Batch 26/26 - Loss: 0.6973 [520/820]
Epoch Training Loss: 0.6949
Test Results:
 Accuracy: 49.8%, Avg Loss: 

['/content/wandb/run-20241122_044045-u8jaqks4/files/NeuralNetworkSigmoid.pth']

Run the 3rd Neural Network:

In [ ]:
model = NeuralNetworkReLUSigmoid().to(device)
optimizer = torch.optim.Adam(model_relu_sigmoid.parameters(), lr=wandb.config.learning_rate)

for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch+1}\n------------------------------")
    train(train_loader, model, loss_func, optimizer)
    test(test_loader, model, loss_func)

torch.save(model.state_dict(), "NeuralNetworkReLUSigmoid.pth")
wandb.save("NeuralNetworkReLUSigmoid.pth")

Epoch 1
------------------------------
Batch 1/26 - Loss: 0.6923 [32/820]
Batch 6/26 - Loss: 0.6927 [192/820]
Batch 11/26 - Loss: 0.6908 [352/820]
Batch 16/26 - Loss: 0.6938 [512/820]
Batch 21/26 - Loss: 0.6933 [672/820]
Batch 26/26 - Loss: 0.6930 [520/820]
Epoch Training Loss: 0.6927
Test Results:
 Accuracy: 50.2%, Avg Loss: 0.6927

Epoch 2
------------------------------
Batch 1/26 - Loss: 0.6911 [32/820]
Batch 6/26 - Loss: 0.6909 [192/820]
Batch 11/26 - Loss: 0.6920 [352/820]
Batch 16/26 - Loss: 0.6925 [512/820]
Batch 21/26 - Loss: 0.6913 [672/820]
Batch 26/26 - Loss: 0.6938 [520/820]
Epoch Training Loss: 0.6927
Test Results:
 Accuracy: 50.2%, Avg Loss: 0.6927

Epoch 3
------------------------------
Batch 1/26 - Loss: 0.6944 [32/820]
Batch 6/26 - Loss: 0.6933 [192/820]
Batch 11/26 - Loss: 0.6932 [352/820]
Batch 16/26 - Loss: 0.6932 [512/820]
Batch 21/26 - Loss: 0.6936 [672/820]
Batch 26/26 - Loss: 0.6916 [520/820]
Epoch Training Loss: 0.6927
Test Results:
 Accuracy: 50.2%, Avg Loss: 

['/content/wandb/run-20241122_044045-u8jaqks4/files/NeuralNetworkReLUSigmoid.pth']

**Conclusion:**

The experiment revealed that none of the models achieved meaningful learning or significant performance improvements. The Tanh-Sigmoid model, which typically supports effective gradient flow, performed the worst with an accuracy of 45.4%. This suggests potential issues with data quality as the architecture itself should theoretically perform better.

The Sigmoid-Sigmoid model achieved an accuracy of 49.8%. Its consistent lack of improvement highlights the limitations of using Sigmoid activations throughout the network, as this likely caused vanishing gradients that hindered effective weight updates.

The ReLU-Sigmoid model showed slightly better performance with an accuracy of 50.2%, but it also failed to learn meaningful patterns. While ReLU addresses vanishing gradients in the hidden layers, the combination of ReLU and Sigmoid activations appears to be ineffective for this particular task.

Overall, the poor performance across all models suggests that external factors, such as data preprocessing, initialization, or optimization settings, may have contributed to these results.